In [4]:
import os
import json
from PIL import Image, UnidentifiedImageError

# Define paths
dataset_dir = r'C:\Users\Sejal Hanmante\OneDrive\Desktop\idd detection\IDD_FGVD'
image_dir = os.path.join(dataset_dir)  # No need to add subfolders, handled inside
annotation_dir = os.path.join(dataset_dir)
label_dir = os.path.join(dataset_dir, 'labels')

# Ensure the label directories exist
os.makedirs(os.path.join(label_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(label_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(label_dir, 'test'), exist_ok=True)

# Define class mappings if applicable
class_map = {'class_name_1': 0, 'class_name_2': 1}  # Example class mapping, adjust as per your classes

# Function to convert JSON annotation to YOLO format
def json_to_yolo(json_path, image_path, label_path):
    try:
        # Open the image and get its dimensions for normalization
        img = Image.open(image_path)
        img_width, img_height = img.size
    except UnidentifiedImageError:
        print(f"Warning: Cannot identify image file {image_path}. Skipping this file.")
        return
    except Exception as e:
        print(f"Error: {e}. Skipping {image_path}.")
        return

    # Load the JSON annotation
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error: Failed to load JSON file {json_path}. Error: {e}. Skipping this file.")
        return

    # Open a new .txt file to write the annotations
    try:
        with open(label_path, 'w') as label_file:
            for obj in data.get('annotations', []):  # Look for 'annotations' key (adjust based on your structure)
                class_name = obj.get('class', '')
                class_id = class_map.get(class_name, None)
                if class_id is None:
                    continue  # Skip classes not in the mapping
                
                # Bounding box information
                xmin, ymin, xmax, ymax = obj.get('bbox', [0, 0, 0, 0])
                
                # Normalize the coordinates
                center_x = (xmin + xmax) / 2 / img_width
                center_y = (ymin + ymax) / 2 / img_height
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height
                
                # Write the annotation to the label file
                label_file.write(f"{class_id} {center_x} {center_y} {width} {height}\n")
    except Exception as e:
        print(f"Error: Failed to write label file {label_path}. Error: {e}.")

# Function to inspect the structure of a sample JSON file
def inspect_json_structure(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    print(json.dumps(data, indent=4))  # Pretty print the JSON to inspect its structure

# Example: Inspect the first JSON file in the 'train' set (You can skip this step once you know the structure)
sample_json_path = os.path.join(annotation_dir, 'train', 'json', os.listdir(os.path.join(annotation_dir, 'train', 'json'))[0])
inspect_json_structure(sample_json_path)

# Loop through splits (train, val, test)
for split in ['train', 'val', 'test']:
    image_split_dir = os.path.join(image_dir, split, 'images')
    annotation_split_dir = os.path.join(annotation_dir, split, 'json')
    
    # Loop through images in the split
    for image_filename in os.listdir(image_split_dir):
        if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
            image_path = os.path.join(image_split_dir, image_filename)
            json_filename = image_filename.replace('.jpg', '.json').replace('.png', '.json')
            json_path = os.path.join(annotation_split_dir, json_filename)
            
            # Ensure the JSON annotation exists
            if os.path.exists(json_path):
                # Label path corresponding to the image
                label_path = os.path.join(label_dir, split, image_filename.replace('.jpg', '.txt').replace('.png', '.txt'))
                
                # Convert JSON annotation to YOLO format
                json_to_yolo(json_path, image_path, label_path)


{
    "image": "IDD_finegrained/Main_part/5206-6234/Images/5393.jpg",
    "annos": [
        {
            "class": "motorcycle",
            "bbox": [
                996.0,
                619.0,
                1086.0,
                752.0
            ]
        },
        {
            "class": "car",
            "bbox": [
                855.6,
                565.0,
                958.9,
                688.63
            ]
        }
    ]
}


#### Model Training

In [1]:
!yolo train model=yolov8n.yaml data="C:/Users/Sejal Hanmante/OneDrive/Desktop/idd detection/IDD_FGVD/dataset.yaml" epochs=50 batch=16 imgsz=640 verbose=True

: 